In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv


In [2]:
# Importing necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras import models, layers, Sequential
from keras.layers.core import  Lambda , Dense, Flatten, Dropout

Using TensorFlow backend.


In [3]:
# Reading in the data

train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
sub = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')

In [4]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_train = (train.iloc[:, 1:].values).astype('float32')
y_train = (train.iloc[:, 0].values).astype('int32')

X_test = (test.values).astype('float32')

In [7]:
# Reshaping X_train, X_test into (# of imgs, 28, 28, 1)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [8]:
# encoding the categorical y_train
from keras.utils import to_categorical
y_train = to_categorical(y_train)

# Standardize the data
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

def standardize(x): 
    return (x-mean_px)/std_px

In [9]:
#  Building the neural network
network = models.Sequential()
network.add(Lambda(standardize,input_shape=(28,28,1)))
network.add(layers.Flatten())
network.add(layers.Dense(512, activation='relu'))
network.add(layers.Dense(512, activation='relu'))
network.add(layers.Dense(10, activation='softmax'))

In [10]:
# compile the network
network.compile(optimizer='Adam',
               loss='categorical_crossentropy',
               metrics = ['accuracy'])

In [11]:
# Splitting the data into training set and test set
from sklearn.model_selection import train_test_split
X = X_train
y = y_train

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [12]:
# fit the training data into network
network.fit(X_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
296/296 [==============================] - 2s 7ms/step - loss: 0.2300 - accuracy: 0.9288
Epoch 2/10
296/296 [==============================] - 2s 7ms/step - loss: 0.0925 - accuracy: 0.9708
Epoch 3/10
296/296 [==============================] - 2s 7ms/step - loss: 0.0563 - accuracy: 0.9819
Epoch 4/10
296/296 [==============================] - 2s 7ms/step - loss: 0.0429 - accuracy: 0.9857
Epoch 5/10
296/296 [==============================] - 2s 7ms/step - loss: 0.0331 - accuracy: 0.9891
Epoch 6/10
296/296 [==============================] - 2s 7ms/step - loss: 0.0297 - accuracy: 0.9901
Epoch 7/10
296/296 [==============================] - 2s 7ms/step - loss: 0.0209 - accuracy: 0.9933
Epoch 8/10
296/296 [==============================] - 2s 6ms/step - loss: 0.0230 - accuracy: 0.9930
Epoch 9/10
296/296 [==============================] - 2s 7ms/step - loss: 0.0204 - accuracy: 0.9933
Epoch 10/10
296/296 [==============================] - 2s 7ms/step - loss: 0.0172 - accuracy: 0.9944

In [13]:
# Evaluating the trained model to our train-test split
network.evaluate(X_val, y_val)

132/132 [==============================] - 0s 2ms/step - loss: 0.1126 - accuracy: 0.9760


[0.11260466277599335, 0.9759523868560791]

In [14]:
# The predictions for the test input
pred = network.predict(X_test)
pred = np.argmax(pred, axis = 1)

In [15]:
pred.shape

(28000,)

In [16]:
# Write the predictions
sub['Label'] = pred

In [17]:
# Write the output file
sub.to_csv('final_sub.csv', index=False, header=True)

In [18]:
df = pd.read_csv('final_sub.csv')

In [19]:
df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
